In [ ]:
# plant1,2의 모든 데이터는 `동일한 시간`에 측정된 자료이다,
# 즉, 24시간, 48시간뒤의 결로 유무에 대한 데이터는 모든 측정이 완료된 이후 정답지의 시간 이전 24시간의 기상 온도, 기상습도를
# 토대로 결로 여부를 측정해야 하는 것이다!

# 방법1 처음부터 기상 온도와 습도를 기준으로 결로 여부를 예측
# 방법2 기상 온도 습도를 토대로 공장내 온도와 공장내 습도를 예측후 예측한 값으로 결로 유무를 예측(2중 예측)
# 방법3 기상온도, 기상 습도 뿐만 아니라 여러 다른 요소들도 포함( 데이콘 온도 습도 예측 모델을 참조 )
# 풍속 풍향은 결로에 영향X, 우천시(습도 상승시)결로 발생 가능성 커짐
# 방법4 단순히 공장내 온도와 습도만으로 결로 현상 예측하기


# 현재는 방법4를 통해 결로 현상 여부의 확률을 측정 중이다(-> 시간이 오래 걸리니 비교군으로 결로 발생 X를 500개, 결로발생 O인 값을 나머지 전부 측정해서 해당 확률을 측정한다.)
# -> 51000건을 예측하는데 7시간 걸림


In [1]:
#기상습도, 기상습도, 공장내 습도, 공장내 온도에서 iloc[2055:2060] 부분을 삭제하라
import pandas as pd
from numpy import mean
from scipy import stats #scipy란 과학기술 통계분석함수 모음이다.
import numpy as np
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'NanumGothicCoding'
plt.rcParams['axes.unicode_minus'] = False
import mglearn 
import missingno as msno
import seaborn as sns
from keras.optimizers import RMSprop
from keras.models import Sequential,load_model
from keras.layers import Dense, Activation,LSTM,Dropout
from keras import layers
from keras.optimizers import RMSprop
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

import keras
from mpl_toolkits.mplot3d import Axes3D
from keras import models
from keras.utils import np_utils
from numpy import argmax

Using TensorFlow backend.


In [198]:
weather_df=pd.read_csv('C:/Anaconda3/python/csvdata/hyundae/gogo/weather_df.csv')                       
plant1_train=pd.read_csv('C:/Anaconda3/python/csvdata/hyundae/plant1_traincp.csv')


In [199]:
weather_df=weather_df.iloc[0:,1:]
plant1_train=plant1_train.iloc[0:,1:]


In [200]:
weather_df

,일시,기온(°C),습도(%)
0,2016-01-01 0:01,-0.3,87.5
1,2016-01-01 0:02,-0.3,87.5
2,2016-01-01 0:03,-0.3,87.6
3,2016-01-01 0:04,-0.4,87.6
4,2016-01-01 0:05,-0.4,87.5
...,...,...,...
2090471,2019-12-31 23:56,-6.3,77.8
2090472,2019-12-31 23:57,-6.3,77.9
2090473,2019-12-31 23:58,-6.3,77.7
2090474,2019-12-31 23:59,-6.4,78.0


In [201]:
plant1_train.columns=plant1_train.columns.str.replace('plant1_train.','')

In [202]:
plant1_train['mea_ddhr'] = pd.to_datetime(plant1_train['mea_ddhr'], format='%Y-%m-%d %H:%M:%S', errors='raise')

#마찬가지로 '일시'도 datetime으로 변환
weather_df['일시'] = pd.to_datetime(weather_df['일시'], format='%Y-%m-%d %H:%M:%S', errors='raise')

In [203]:
plant1_train=plant1_train.rename(columns={'mea_ddhr':'일시'})

In [204]:
#결측치를 제거한다.
plant1_train.isna().sum()
plant1_train=plant1_train.dropna(axis=0)
plant1_train.shape

(57854, 15)

In [205]:
weather_df=weather_df.dropna(axis=0)
weather_df

,일시,기온(°C),습도(%)
0,2016-01-01 00:01:00,-0.3,87.5
1,2016-01-01 00:02:00,-0.3,87.5
2,2016-01-01 00:03:00,-0.3,87.6
3,2016-01-01 00:04:00,-0.4,87.6
4,2016-01-01 00:05:00,-0.4,87.5
...,...,...,...
2090471,2019-12-31 23:56:00,-6.3,77.8
2090472,2019-12-31 23:57:00,-6.3,77.9
2090473,2019-12-31 23:58:00,-6.3,77.7
2090474,2019-12-31 23:59:00,-6.4,78.0


In [206]:
df_plant1=pd.merge(weather_df, plant1_train, on='일시')

In [207]:
df_plant1=df_plant1.dropna(axis=0)
df_plant1.isna().sum()
#결측치 제거 및 '일시'와 'mea_ddhr'이 동일한 값들만 남았다.
df_plant1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57655 entries, 0 to 57654
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   일시             57655 non-null  datetime64[ns]
 1   기온(°C)         57655 non-null  float64       
 2   습도(%)          57655 non-null  float64       
 3   tem_in_loc1    57655 non-null  float64       
 4   hum_in_loc1    57655 non-null  float64       
 5   tem_coil_loc1  57655 non-null  float64       
 6   tem_in_loc2    57655 non-null  float64       
 7   hum_in_loc2    57655 non-null  float64       
 8   tem_coil_loc2  57655 non-null  float64       
 9   tem_in_loc3    57655 non-null  float64       
 10  hum_in_loc3    57655 non-null  float64       
 11  tem_coil_loc3  57655 non-null  float64       
 12  tem_out_loc1   57655 non-null  float64       
 13  hum_out_loc1   57655 non-null  float64       
 14  cond_loc1      57655 non-null  float64       
 15  cond_loc2      5765

In [210]:
plant1_tem_hum=df_plant1[['일시','기온(°C)','습도(%)','tem_in_loc1','hum_in_loc1','tem_coil_loc1','cond_loc1']]

plant1_tem_hum.index=plant1_tem_hum['일시']